### <center>Libraries and Configs</center>

In [19]:
# Import libraries
import os
from glob import glob
import json
import pandas as pd
pd.set_option('display.max_columns', None)
from tqdm import tqdm

In [2]:
# Declare dataset configs
dataset_configs = {
    'train_metadata_path': r'..\datasets\train.csv',
    'train_tracking_dir_path': r'..\datasets\train_tracking',
    'train_annotation_dir_path': r'..\datasets\train_annotation',
    'test_metadata_path': r'..\datasets\test.csv',
    'test_tracking_dir_path': r'..\datasets\test_tracking',
    'sample_submission_file_path': r'..\datasets\sample_submission.csv'
}

### <center>Filter Data</center>

In [ ]:
# Get all tracking and annotation files (recursive)
tracking_files = glob(os.path.join(dataset_configs['train_tracking_dir_path'], '**', '*.parquet'), recursive=True)
annotation_files = glob(os.path.join(dataset_configs['train_annotation_dir_path'], '**', '*.parquet'), recursive=True)
print("Number of tracking files:", len(tracking_files))
print("Number of annotation files:", len(annotation_files))


Number of tracking files: 8789
Number of annotation files: 847


In [13]:
# Get file ids with both tracking and annotation files
tracking_file_ids = {os.path.splitext(os.path.basename(f))[0] for f in tracking_files}
annotation_file_ids = {os.path.splitext(os.path.basename(f))[0] for f in annotation_files}
valid_video_ids = tracking_file_ids.intersection(annotation_file_ids)
valid_video_ids = [int(vid) for vid in valid_video_ids]
print("Number of valid video ids with both tracking and annotation files:", len(valid_video_ids))

Number of valid video ids with both tracking and annotation files: 847


In [14]:
# Read metadata file
metadata_df = pd.read_csv(dataset_configs['train_metadata_path'])
metadata_df.head()


,lab_id,video_id,mouse1_strain,mouse1_color,mouse1_sex,mouse1_id,mouse1_age,mouse1_condition,mouse2_strain,mouse2_color,mouse2_sex,mouse2_id,mouse2_age,mouse2_condition,mouse3_strain,mouse3_color,mouse3_sex,mouse3_id,mouse3_age,mouse3_condition,mouse4_strain,mouse4_color,mouse4_sex,mouse4_id,mouse4_age,mouse4_condition,frames_per_second,video_duration_sec,pix_per_cm_approx,video_width_pix,video_height_pix,arena_width_cm,arena_height_cm,arena_shape,arena_type,body_parts_tracked,behaviors_labeled,tracking_method
0,AdaptableSnail,44566106,CD-1 (ICR),white,male,10.0,8-12 weeks,wireless device,CD-1 (ICR),white,male,24.0,8-12 weeks,wireless device,CD-1 (ICR),white,male,38.0,8-12 weeks,wireless device,CD-1 (ICR),white,male,51.0,8-12 weeks,wireless device,30.0,615.6,16.0,1228,1068,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""head...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
1,AdaptableSnail,143861384,CD-1 (ICR),white,male,3.0,8-12 weeks,NaN,CD-1 (ICR),white,male,17.0,8-12 weeks,NaN,CD-1 (ICR),white,male,31.0,8-12 weeks,NaN,CD-1 (ICR),white,male,44.0,8-12 weeks,NaN,25.0,3599.0,9.7,968,608,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
2,AdaptableSnail,209576908,CD-1 (ICR),white,male,7.0,8-12 weeks,NaN,CD-1 (ICR),white,male,21.0,8-12 weeks,NaN,CD-1 (ICR),white,male,35.0,8-12 weeks,NaN,CD-1 (ICR),white,male,48.0,8-12 weeks,NaN,30.0,615.2,16.0,1266,1100,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
3,AdaptableSnail,278643799,CD-1 (ICR),white,male,11.0,8-12 weeks,wireless device,CD-1 (ICR),white,male,25.0,8-12 weeks,wireless device,CD-1 (ICR),white,male,39.0,8-12 weeks,wireless device,NaN,NaN,NaN,NaN,NaN,NaN,30.0,619.7,16.0,1224,1100,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""head...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
4,AdaptableSnail,351967631,CD-1 (ICR),white,male,14.0,8-12 weeks,NaN,CD-1 (ICR),white,male,28.0,8-12 weeks,NaN,CD-1 (ICR),white,male,42.0,8-12 weeks,NaN,NaN,NaN,NaN,NaN,8-12 weeks,NaN,30.0,602.6,16.0,1204,1068,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut


In [15]:
# Filter out only valid metadata
metadata_df = metadata_df[metadata_df['video_id'].isin(valid_video_ids)].reset_index(drop=True)
print("Filtered metadata shape:", metadata_df.shape)
metadata_df.head()

Filtered metadata shape: (847, 38)


,lab_id,video_id,mouse1_strain,mouse1_color,mouse1_sex,mouse1_id,mouse1_age,mouse1_condition,mouse2_strain,mouse2_color,mouse2_sex,mouse2_id,mouse2_age,mouse2_condition,mouse3_strain,mouse3_color,mouse3_sex,mouse3_id,mouse3_age,mouse3_condition,mouse4_strain,mouse4_color,mouse4_sex,mouse4_id,mouse4_age,mouse4_condition,frames_per_second,video_duration_sec,pix_per_cm_approx,video_width_pix,video_height_pix,arena_width_cm,arena_height_cm,arena_shape,arena_type,body_parts_tracked,behaviors_labeled,tracking_method
0,AdaptableSnail,44566106,CD-1 (ICR),white,male,10.0,8-12 weeks,wireless device,CD-1 (ICR),white,male,24.0,8-12 weeks,wireless device,CD-1 (ICR),white,male,38.0,8-12 weeks,wireless device,CD-1 (ICR),white,male,51.0,8-12 weeks,wireless device,30.0,615.6,16.0,1228,1068,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""head...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
1,AdaptableSnail,143861384,CD-1 (ICR),white,male,3.0,8-12 weeks,NaN,CD-1 (ICR),white,male,17.0,8-12 weeks,NaN,CD-1 (ICR),white,male,31.0,8-12 weeks,NaN,CD-1 (ICR),white,male,44.0,8-12 weeks,NaN,25.0,3599.0,9.7,968,608,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
2,AdaptableSnail,209576908,CD-1 (ICR),white,male,7.0,8-12 weeks,NaN,CD-1 (ICR),white,male,21.0,8-12 weeks,NaN,CD-1 (ICR),white,male,35.0,8-12 weeks,NaN,CD-1 (ICR),white,male,48.0,8-12 weeks,NaN,30.0,615.2,16.0,1266,1100,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
3,AdaptableSnail,278643799,CD-1 (ICR),white,male,11.0,8-12 weeks,wireless device,CD-1 (ICR),white,male,25.0,8-12 weeks,wireless device,CD-1 (ICR),white,male,39.0,8-12 weeks,wireless device,NaN,NaN,NaN,NaN,NaN,NaN,30.0,619.7,16.0,1224,1100,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""head...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut
4,AdaptableSnail,351967631,CD-1 (ICR),white,male,14.0,8-12 weeks,NaN,CD-1 (ICR),white,male,28.0,8-12 weeks,NaN,CD-1 (ICR),white,male,42.0,8-12 weeks,NaN,NaN,NaN,NaN,NaN,8-12 weeks,NaN,30.0,602.6,16.0,1204,1068,60.0,60.0,square,familiar,"[""body_center"", ""ear_left"", ""ear_right"", ""late...","[""mouse1,mouse2,approach"", ""mouse1,mouse2,atta...",DeepLabCut


### <center>Data Preparation</center>

In [20]:
# Create a map of video id and labels
actions = {}
for idx, row in metadata_df.iterrows():
    lab_id, video_id = row['lab_id'],row['video_id']
    annotation_df = pd.read_parquet(os.path.join(dataset_configs['train_annotation_dir_path'], lab_id, f"{video_id}.parquet"))
    for action in annotation_df['action'].unique():
        if action not in actions:
            actions[action] = [video_id]
        else:
            actions[action].append(video_id)

print("Actions found: ")
for action, vids in actions.items():
    print(f"Action: {action}, Number of videos: {len(vids)}")

os.makedirs('dumps', exist_ok=True)
with open('dumps/action_video_map.json', 'w') as f:
    json.dump(actions, f, indent=4)

Actions found: 
Action: rear, Number of videos: 85
Action: avoid, Number of videos: 17
Action: attack, Number of videos: 332
Action: approach, Number of videos: 159
Action: chase, Number of videos: 60
Action: submit, Number of videos: 9
Action: chaseattack, Number of videos: 13
Action: shepherd, Number of videos: 8
Action: sniff, Number of videos: 594
Action: mount, Number of videos: 247
Action: disengage, Number of videos: 20
Action: selfgroom, Number of videos: 81
Action: sniffgenital, Number of videos: 405
Action: sniffface, Number of videos: 108
Action: sniffbody, Number of videos: 109
Action: dominancemount, Number of videos: 63
Action: attemptmount, Number of videos: 42
Action: intromit, Number of videos: 81
Action: genitalgroom, Number of videos: 17
Action: reciprocalsniff, Number of videos: 21
Action: escape, Number of videos: 93
Action: dominance, Number of videos: 6
Action: allogroom, Number of videos: 17
Action: ejaculate, Number of videos: 3
Action: defend, Number of videos

In [21]:
pd.read_parquet(annotation_files[0]).head()

,agent_id,target_id,action,start_frame,stop_frame
0,1,3,chase,2,54
1,1,3,chase,128,234
2,3,2,avoid,324,342
3,3,1,avoid,324,342
4,1,2,chase,942,1052


In [24]:
# Agent and target are same
action_to_itself = {}
for idx, row in tqdm(metadata_df.iterrows(), total=metadata_df.shape[0], desc="Checking actions where agent and target is same"):
    lab_id, video_id = row['lab_id'],row['video_id']
    annotation_df = pd.read_parquet(os.path.join(dataset_configs['train_annotation_dir_path'], lab_id, f"{video_id}.parquet"))
    for _, ann_row in annotation_df.iterrows():
        if ann_row['agent_id'] == ann_row['target_id']:
            action = ann_row['action']
            if action not in action_to_itself:
                action_to_itself[action] = [video_id]
            else:
                if video_id not in action_to_itself[action]:
                    action_to_itself[action].append(video_id)

print("Actions where agent and target are same: ")
for action, vids in action_to_itself.items():
    print(f"Action: {action}, Number of videos: {len(vids)}")

os.makedirs('dumps', exist_ok=True)
with open('dumps/action_to_itself_video_map.json', 'w') as f:
    json.dump(action_to_itself, f, indent=4)

Checking actions where agent and target is same: 100%|██████████| 847/847 [00:02<00:00, 314.68it/s]

Actions where agent and target are same: 
Action: rear, Number of videos: 85
Action: selfgroom, Number of videos: 81
Action: genitalgroom, Number of videos: 17
Action: climb, Number of videos: 20
Action: dig, Number of videos: 37
Action: rest, Number of videos: 13
Action: run, Number of videos: 11
Action: freeze, Number of videos: 9
Action: exploreobject, Number of videos: 10
Action: biteobject, Number of videos: 10
Action: huddle, Number of videos: 11


In [25]:
# Action and target are not same
action_to_others = {}
for idx, row in tqdm(metadata_df.iterrows(), total=metadata_df.shape[0], desc="Checking actions where agent and target are different"):
    lab_id, video_id = row['lab_id'],row['video_id']
    annotation_df = pd.read_parquet(os.path.join(dataset_configs['train_annotation_dir_path'], lab_id, f"{video_id}.parquet"))
    for _, ann_row in annotation_df.iterrows():
        if ann_row['agent_id'] != ann_row['target_id']:
            action = ann_row['action']
            if action not in action_to_others:
                action_to_others[action] = [video_id]
            else:
                if video_id not in action_to_others[action]:
                    action_to_others[action].append(video_id)

print("Actions where agent and target are different: ")
for action, vids in action_to_others.items():
    print(f"Action: {action}, Number of videos: {len(vids)}")

os.makedirs('dumps', exist_ok=True)
with open('dumps/action_to_others_video_map.json', 'w') as f:
    json.dump(action_to_others, f, indent=4)

Checking actions where agent and target are different: 100%|██████████| 847/847 [00:02<00:00, 285.27it/s]

Actions where agent and target are different: 
Action: avoid, Number of videos: 17
Action: attack, Number of videos: 332
Action: approach, Number of videos: 159
Action: chase, Number of videos: 60
Action: submit, Number of videos: 9
Action: chaseattack, Number of videos: 13
Action: shepherd, Number of videos: 8
Action: sniff, Number of videos: 594
Action: mount, Number of videos: 247
Action: disengage, Number of videos: 20
Action: sniffgenital, Number of videos: 405
Action: sniffface, Number of videos: 108
Action: sniffbody, Number of videos: 109
Action: dominancemount, Number of videos: 63
Action: attemptmount, Number of videos: 42
Action: intromit, Number of videos: 81
Action: reciprocalsniff, Number of videos: 21
Action: escape, Number of videos: 93
Action: dominance, Number of videos: 6
Action: allogroom, Number of videos: 17
Action: ejaculate, Number of videos: 3
Action: defend, Number of videos: 52
Action: dominancegroom, Number of videos: 14
Action: flinch, Number of videos: 11


Some actions are only done to itself while some actions can only be performed to others. So, the models should be built accordingly.

In [27]:
# Do all frames need to be annotated?

In [26]:
# In same span can a agent and target have multiple actions? Check for one-to-one and one-to-others

In [28]:
# Check which body parts have least representations?

In [29]:
# Check how normalization can be done?